**CSI5138F - Intro to DL and RL - Project**

This notebook is heavily influenced by the following Tutorial:
  https://mccormickml.com/2019/07/22/BERT-fine-tuning/ 

# Set up the notebook

In [ ]:
!pip install --upgrade pip

In [ ]:
# install pytorch transformers
!pip install transformers
#!pip install pytorch_transformers
#!pip install fairseq

## Imports and mount drive

In [ ]:
import torch
import os
import string
import copy
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
#from pytorch_transformers import *
from transformers import XLMRobertaForMaskedLM, XLMRobertaTokenizer
#from fairseq import *
import numpy as np
import json
import collections
#from google.colab import drive
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

# Import nltk WordNet
#import nltk
#nltk.download('wordnet')
#lemmatizer = nltk.stem.WordNetLemmatizer()

# Mount google drive containing the datasets
#drive.mount('/content/drive', force_remount=True)

In [ ]:
#from fairseq.models.roberta import XLMRModel

## Get RoBERTa tokenizer

In [ ]:
tokenizer =XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")

In [ ]:
tokenizer.encode('talking', add_special_tokens=False)

In [ ]:
tok = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")
tok.convert_ids_to_tokens([459, 6496])

## Utility functions

In [ ]:
def load_json_objects_from_file(filename):
  json_objects = []
  with open(filename, mode = "r") as jsonl_file:
      for line in jsonl_file:
          json_objects.append(json.loads(line))
  return json_objects

#Takes a list of words (strings) and a sentence (as a RoBERTa tokenized ID list) and returns a list
#of pairs indicating the tokens' start and end positions in the sentence for each word
#NOTE: it can also apply to a group of words separated by spaces
#NOTE: It is important that the list of words given describes a sentence, because the order is relevant to do the matching properly
      
def find_word_in_tokenized_sentence(word,token_ids, wic_sentence, wic_idx):
  #print(f"token_ids===={token_ids}")
  a = tokenizer.encode(word, add_special_tokens=False)
  decomposedWord = []
  #print(f"token_ids===={decomposedWord}")
  for x in a:
    if x == 0 or x == 2 or x==5 or x == 6:
        continue
    decomposedWord.append(x)
  if isinstance(decomposedWord, int):
    decomposedWord = [decomposedWord]
  #decomposedWord = [x if x != '<s>' and x != '</s>' for x in decomposedWord]
  for i in range(len(token_ids)):
    #print(f"token_ids===={token_ids[i:i+len(decomposedWord)]}+decomposedWord===={decomposedWord}")
    #print(decomposedWord)
    if token_ids[i] == decomposedWord[0] and token_ids[i:min(i+len(decomposedWord), len(token_ids))] == decomposedWord:
      #print('gooodooooooooooooooooood')
      return (i,i+len(decomposedWord)-1)
  # This is the ouput when no matching pattern is found
    #print("fuck")
  print(wic_idx)
  print(f"{tokenizer.encode(word)}_in_{token_ids}")
  print(f"word====={word}+sentence===={wic_sentence}")
  print(f"sentence_decoded===={tok.convert_ids_to_tokens(token_ids)}+++word_decoded{tok.convert_ids_to_tokens(decomposedWord)}")
  return (-1,-1)
  
def find_words_in_tokenized_sentences(wordList,token_ids, wic_sentence, wic_idx):
  intList = []
  for word in wordList:
    if len(intList) == 0:
      #print(find_word_in_tokenized_sentence(word,token_ids))
      try:
          intList.append(find_word_in_tokenized_sentence(word,token_ids, wic_sentence, wic_idx))
      except:
        print(f"{wic_idx}---nothing_found")
        interv = (-1, -1)
    else:
      afterLastInterval = intList[-1][1]+1
      try:
          interv = find_word_in_tokenized_sentence(word,token_ids[afterLastInterval:], wic_sentence, wic_idx)
      except:
        interv = (-1, -1)
        print(f"{wic_idx}---nothing_found")
      #assert(interv == (-1, -1))
      #print(interv)
      actualPositions = (interv[0] + afterLastInterval,interv[1]+afterLastInterval)
      intList.append(actualPositions)
  return intList

def last_find_words_in_tokenized_sentences(word_loc1, word_loc2, token_ids, wic_sentence, wic_idx):
    if wic_idx == 70084 or wic_idx == 25941:
        print(f"{token_ids}=========={wic_sentence}")
    #print(wic_idx)
    intList = []
    sen_split = wic_sentence.split('<s>')
    #print(wic_sentence)
    #word_loc1[0] += 3
    #word_loc1[1] += 3
    word_loc1 = (word_loc1[0] + 4, word_loc1[1] + 4)
    #word_loc2[0] += len(sen_split[1]) + 6
    #word_loc2 += len(sen_split[1]) + 6
    word_loc2 = (word_loc2[0] + 8 + len(sen_split[1]), word_loc2[1] + 8 + len(sen_split[1]))
    if wic_idx == 70084 or wic_idx == 25941:
        print(f"{wic_sentence[word_loc1[0]:word_loc1[1]]}aaaaaandddddd{wic_sentence[word_loc2[0]:word_loc2[1]]}")
    decoded = tokenizer.convert_ids_to_tokens(token_ids)
    if wic_idx == 70084 or wic_idx == 25941:
        print(f"{word_loc1}_and_{word_loc2}___in_________________---------{decoded}")
    sloc1, eloc1, sloc2, eloc2 = 0,0,0,0
    cnt = 0
    word1pos=[]
    word2pos=[]
    for i, sub in enumerate(decoded):
        if cnt >= word_loc2[1]:
            break
        if cnt + len(sub) < word_loc1[0]:
            cnt += len(sub)
            continue
        if cnt >= word_loc1[0] and cnt < word_loc1[1]:
            word1pos.append(i)
        elif cnt >= word_loc2[0] and cnt < word_loc2[1]:
            word2pos.append(i)
        elif cnt + len(sub) >= word_loc1[0] and cnt + len(sub) < word_loc1[1]:
            word1pos.append(i)
        elif cnt + len(sub) >= word_loc2[0] and cnt + len(sub) < word_loc2[1]:
            word2pos.append(i)
        elif cnt < word_loc1[0] and cnt + len(sub) >= word_loc1[1]:
            word1pos.append(i)
        elif cnt < word_loc2[0] and cnt + len(sub) >= word_loc2[1]:
            word2pos.append(i)
        cnt += len(sub)
        continue
        #if cnt + len(sub) < word_loc1[0]:
        #    cnt += len(sub)
        #    continue
        #if cnt + len(sub) < word_loc2[0]:
        #    cnt += len(sub)
        #    continue
        #if
    if wic_idx == 70084 or wic_idx == 25941:
        print(f"{token_ids[min(word1pos):max(word1pos) + 1]}anddddddddddddddddd{token_ids[min(word2pos):max(word2pos) + 1]}")
    return [(min(word1pos), max(word1pos)), (min(word2pos), max(word2pos))]
            
def new_find_words_in_tokenized_sentences(word_loc1, word_loc2, token_ids, wic_sentence, wic_idx):
    intList = []
    sen_split = wic_sentence.split('<s>')
    len1 = len(sen_split[0])
    part1 = tokenizer.encode(wic_sentence[:word_loc1[0] + 3].rstrip(), add_special_tokens=False)
    part2 = tokenizer.encode(' ' + wic_sentence[word_loc1[0] + 3:word_loc1[1] + 3], add_special_tokens=False)
    part3 = tokenizer.encode(wic_sentence[word_loc1[1] + 3: len1 + word_loc2[0] + 6].rstrip(), add_special_tokens=False)
    part4 = tokenizer.encode(' ' + wic_sentence[len1 + word_loc2[0] + 6: len1 + word_loc2[1] + 6], add_special_tokens=False)
    part5 = tokenizer.encode(wic_sentence[len1 + word_loc2[1] + 6:], add_special_tokens=False)
    print(f"Sentence converted=={tokenizer.convert_ids_to_tokens(token_ids)}+ part1===={tokenizer.convert_ids_to_tokens(part1)}")
    print(f"part1=={part1}+sentence{token_ids}")
    spos1 = 0
    epos1 = 0
    spos2, epos2 = 0, 0
    while len(part1) > 0 and token_ids[0] == part1[0]:
        token_ids.pop(0)
        part1.pop(0)
        spos1 += 1
        spos2 += 1
    if len(part1) > 0:
        print("Not part1")
        print(wic_idx)
        print(wic_sentence)
    epos1 = spos1
    print(part3)
    print(token_ids)
    while part3[0] != token_ids[0]:
        token_ids.pop(0)
        epos1 += 1
    spos2 = epos1
    while len(part3) > 0 and token_ids[0] == part3[0]:
        token_ids.pop(0)
        part3.pop(0)
        spos2 += 1
    epos2 = spos2
    print(f"part5=={part5}+decoded{tokenizer.convert_ids_to_tokens(part5)}")
    while part5[0] != token_ids[0]:
        token_ids.pop(0)
        epos2 += 1
    return [(spos1, epos1), (spos2, epos2)]
    
    
    

def flat_accuracy(preds, labels, return_predict_correctness = False):
  pred_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()
  if return_predict_correctness:
    return np.sum(pred_flat == labels_flat) / len(labels_flat), pred_flat == labels_flat
  else:
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def flat_predictions(preds):
  pred_flat = np.argmax(preds, axis=1).flatten()
  return pred_flat == 1

## Constants

In [ ]:
BATCH_SIZE = 32
EPOCHS = 80
PATIENCE = 6
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

In [ ]:
PATIENCE = 7

In [ ]:
from tqdm import tqdm

## Import and Process the WordInContext Training data

In [ ]:
# Create a function to preprocess the WiC data
bad_arr = []
idx_arr = []
bad_objects = []
def wic_preprocessing(json_objects, training = True, shuffle_data = False, verbose = False):
  wic_sentences = []
  wic_encoded = []
  wic_labels = []
  wic_word_locs = []
  wic_indexes = []
  for index, example in tqdm(enumerate(json_objects)):
    if example['idx'] in [8761, 14399, 3460, 14070, 35431, 34543]:
        continue
    #print(index)
    #if index == 86176:
    #    print(example)
    if example['idx'] == 70084:
        continue
    wic_indexes.append(example['idx']) 
    #wic_indexes.append(index)
    #print(example['sentence1'])
    sentence = f"<s>{example['sentence1']}</s><s>{example['sentence2']}</s>"
    if len(sentence) > 1000:
        bad_objects.append(example)
        bad_arr.append(len(sentence))
        idx_arr.append(example['idx'])
    #if index == 14934:
    #    print(sentence)
    wic_sentences.append(sentence)
    # encoded the sentences
    wic_encoded.append(tokenizer.encode(sentence, add_special_tokens=False))
    word = example['word']
    word_locs = (-1, -1)
    sent1_split = example['sentence1'].split(' ')
    sent2_split = example['sentence2'].split(' ')
    #sent1_split = ['_' + x for x in sent1_split]
    #sent2_split = ['_' + x for x in sent2_split]
    
    # Get the index of word in both sentences
    sent1_word_char_loc = (example['start1'], example['end1'])
    sent2_word_char_loc = (example['start2'], example['end2'])
    # Create a variable to keep track of the number of characters parsed in each sentence as we loop
    sent_chars = 0
    i, j = 0, 0
    word1_not_found, word2_not_found = True, True
    while word1_not_found and i < len(sent1_split):
      word_len = len(sent1_split[i])
      if sent_chars >= sent1_word_char_loc[0] or sent_chars + word_len >= sent1_word_char_loc[1]:
        word_locs = (i, -1) # Found the word in the sentence
        word1_not_found = False
      elif sent_chars > sent1_word_char_loc[1]:
        # If we got past the word. Assume it was the previous word
        word_locs = (i - 1, -1) # Found the word in the sentence
        word1_not_found = False
      else:
        # Look at the next word
        sent_chars += word_len + 1 # Plus one for the space
        i += 1
    # Loop over the words in the second
    sent_chars = 0 # Reset
    while word2_not_found and j < len(sent2_split):
      word_len = len(sent2_split[j])
      if sent_chars >= sent2_word_char_loc[0] or sent_chars + word_len >= sent2_word_char_loc[1]:
        word_locs = (i, j) # Found the word in the sentence
        word2_not_found = False
      elif sent_chars > sent2_word_char_loc[1]:
        # If we somehow got past the word. Assume it was the previous word
        word_locs = (i, j - 1) # Found the word in the sentence
        word2_not_found = False
      else:
        # Look at the next word
        sent_chars += word_len + 1 # Plus one for the space
        j += 1
    # For testing
    if verbose:
      print(word)
      print(sent1_split)
      print(sent2_split)
      print(word_locs)
    # Now to find the word in the tokenized sentences
    word1 = example['sentence1'][example['start1']:example['end1']]#[#sent1_split[word_locs[0]]#.translate(str.maketrans('', '', string.punctuation)) #Remove punctuation
    word2 = example['sentence2'][example['start2']:example['end2']]#sent2_split[word_locs[1]]#.translate(str.maketrans('', '', string.punctuation)) #Remove punctuation
    #print(word1, word2)
    token_word_locs = last_find_words_in_tokenized_sentences(sent1_word_char_loc, sent2_word_char_loc, wic_encoded[-1], wic_sentences[-1], wic_indexes[-1])
    wic_word_locs.append(token_word_locs)
    # Get the label if we expect it to be there
    if training:
      if example['label']:
        wic_labels.append(1)
      else:
        wic_labels.append(0)
  # Pad the sequences and find the encoded word location in the combined input
  max_len = np.array([len(ex) for ex in wic_encoded]).max()
  wic_padded = {"input_ids" : [], "attention_mask" : [], "token_type_ids" : [], "word1_locs": [], "word2_locs" : [], "index" : wic_indexes}
  for i in tqdm(range(0, len(wic_encoded))):
    enc_sentence = wic_encoded[i]
    word_locs = wic_word_locs[i]
    # Pad the sequences
    ex_len = len(enc_sentence)
    padded_sentence = enc_sentence
    padded_sentence.extend([0]*(max_len - ex_len))
    wic_padded["input_ids"].append(padded_sentence)
    padded_mask = [1] * ex_len
    padded_mask.extend([0]*(max_len - ex_len))
    wic_padded["attention_mask"].append(padded_mask)
    token_word_locs = wic_word_locs[i]
    first_word_loc = []
    second_word_loc = []
    len_first_word = token_word_locs[0][1] - token_word_locs[0][0] + 1
    len_second_word = token_word_locs[1][1] - token_word_locs[1][0] + 1
    for j in range(0, max_len):
      if j >= token_word_locs[0][0] and j <= token_word_locs[0][1]:
        #Part of the first word
        first_word_loc.append(1.0)# / len_first_word)
      else:
        first_word_loc.append(0.0)
      if j >= token_word_locs[1][0] and j <= token_word_locs[1][1]:
        #Part of the second word
        second_word_loc.append(1.0)# / len_second_word)
      else:
        second_word_loc.append(0.0)
    # We want to append a [1, max_len] vector instead of a [max_len] vector so wrap in an array
    wic_padded["word1_locs"].append([first_word_loc])
    wic_padded["word2_locs"].append([second_word_loc])
    # token_type_ids is a mask that tells where the first and second sentences are
    token_type_id = []
    first_sentence = True
    sentence_start = True
    for token in padded_sentence:
      if first_sentence and sentence_start and token == 0:
        # Allows 0 at the start of the first sentence
        token_type_id.append(0)
      elif first_sentence and token > 0:
        if sentence_start:
          sentence_start = False
        token_type_id.append(0)
      elif first_sentence and not sentence_start and token == 0:
        first_sentence = False
        # Start of second sentence
        token_type_id.append(1)
      else:
        # Second sentence
        token_type_id.append(1)
    wic_padded["token_type_ids"].append(token_type_id)
  if training:
    if shuffle_data:
      # Shuffle the data
      raw_set = {"input_ids": [], "token_type_ids": [], "attention_mask": [], "labels": [], "word1_locs": [], "word2_locs" : [], "index" : []}
      raw_set["input_ids"], raw_set["token_type_ids"], raw_set["attention_mask"], raw_set["labels"], raw_set["word1_locs"], raw_set["word2_locs"], raw_set["index"] = shuffle(
          wic_padded["input_ids"], wic_padded["token_type_ids"], wic_padded["attention_mask"], wic_labels, wic_padded["word1_locs"], wic_padded["word2_locs"], wic_padded["index"])
    else:
      raw_set = {"input_ids": wic_padded["input_ids"], "token_type_ids": wic_padded["token_type_ids"],
                 "attention_mask": wic_padded["attention_mask"], "labels": wic_labels, "index" : wic_padded["index"],
                 "word1_locs": wic_padded["word1_locs"], "word2_locs" : wic_padded["word2_locs"]}
  else: # No labels present (Testing set)
    # Do not shuffle the testing set
    raw_set = {"input_ids": wic_padded["input_ids"], "token_type_ids": wic_padded["token_type_ids"], 
               "attention_mask": wic_padded["attention_mask"], "index" : wic_padded["index"], 
               "word1_locs": wic_padded["word1_locs"], "word2_locs" : wic_padded["word2_locs"]}
  # Return the raw data (Need to put them in a PyTorch tensor and dataset)
  return raw_set

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train, test = train_test_split(train_json_objs, test_size=5000, random_state=8)

In [ ]:
len(test)

In [ ]:
# Process the data
train_json_objs = load_json_objects_from_file("../input/5-mcl-wic/mcl_wic_new/Russe_Wic_MCL_wic_XL-Ch_Xl-fr.jsonl")
train, test = train_test_split(train_json_objs, test_size=23000, random_state=8)
raw_train_set = wic_preprocessing(train, shuffle_data=False, verbose = False) # We do not want to shuffle for now.
print(train_json_objs[raw_train_set["index"][15]])
print(raw_train_set["input_ids"][15]),
print(raw_train_set["token_type_ids"][15]),
print(raw_train_set["attention_mask"][15]),
print(raw_train_set["labels"][15])
print(raw_train_set["word1_locs"][15]) #right, so it's better to use one-hot vectors to do a selection afterwards, I can compute it more precisely with the token intervals however
print(raw_train_set["word2_locs"][15])

In [ ]:
bad_arr

In [ ]:
idx_arr

In [ ]:
bad_objects

In [ ]:
len(train)

In [ ]:
# Process the data
train_json_objs = load_json_objects_from_file("../input/wic-second/WiC/train.jsonl")
raw_train_set = wic_preprocessing(train_json_objs, shuffle_data=True, verbose = False) # We do not want to shuffle for now.
print(train_json_objs[raw_train_set["index"][15]])
print(raw_train_set["input_ids"][15]),
print(raw_train_set["token_type_ids"][15]),
print(raw_train_set["attention_mask"][15]),
print(raw_train_set["labels"][15])
print(raw_train_set["word1_locs"][15]) #right, so it's better to use one-hot vectors to do a selection afterwards, I can compute it more precisely with the token intervals however
print(raw_train_set["word2_locs"][15])

In [ ]:
print(len(raw_train_set["labels"])/BATCH_SIZE)

### Load into a PyTorch dataset

In [ ]:

train_data = TensorDataset(
    torch.tensor(raw_train_set["input_ids"]),
    torch.tensor(raw_train_set["token_type_ids"]),
    torch.tensor(raw_train_set["attention_mask"]),
    torch.tensor(raw_train_set["labels"]),
    torch.tensor(raw_train_set["word1_locs"]),
    torch.tensor(raw_train_set["word2_locs"]),
    torch.tensor(raw_train_set["index"])
)

train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

### Repeat the procedure to load the WiC validation and testing sets

In [ ]:
# Load the json objects from each file
#test_json_objs = load_json_objects_from_file("../input/mcl-wic-1/mcl_wic_json/test.jsonl")[0]
#print(test_json_objs)
#valid_json_objs = load_json_objects_from_file("../input/mcl-wic-1/mcl_wic_json/val.jsonl")[0]
# Process the objects
print(len(test))
#raw_test_set = wic_preprocessing(test_json_objs, training = False, verbose=False) # The labels for the testing set are unknown
raw_valid_set = wic_preprocessing(test)
# Create PyTorch datasets
#test_data = TensorDataset(
#    torch.tensor(raw_test_set["input_ids"]),
#    torch.tensor(raw_test_set["token_type_ids"]),
#    torch.tensor(raw_test_set["attention_mask"]),
#    torch.tensor(raw_test_set["word1_locs"]),
#    torch.tensor(raw_test_set["word2_locs"]),
#    torch.tensor(raw_test_set["index"])
#)
validation_data = TensorDataset(
    torch.tensor(raw_valid_set["input_ids"]),
    torch.tensor(raw_valid_set["token_type_ids"]),
    torch.tensor(raw_valid_set["attention_mask"]),
    torch.tensor(raw_valid_set["labels"]),
    torch.tensor(raw_valid_set["word1_locs"]),
    torch.tensor(raw_valid_set["word2_locs"]),
    torch.tensor(raw_valid_set["index"])
)
# Create a sampler and loader for each
#test_sampler = RandomSampler(test_data)
#test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=4)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=4)

In [ ]:
bad_arr

In [ ]:
idx_arr

In [ ]:
print(len(raw_valid_set["labels"])/4)

# RoBERTa

## Loading the model

In [ ]:

model = XLMRobertaForMaskedLM.from_pretrained('xlm-roberta-base')
#roberta_init_weights = model.state_dict()

## Create a custom head for WiC instead of a classification head
Based on https://pytorch.org/tutorials/beginner/examples_nn/two_layer_net_module.html and https://huggingface.co/transformers/_modules/transformers/modeling_roberta.html#RobertaModel



In [ ]:
class WiC_Head(torch.nn.Module):
    def __init__(self, roberta_based_model, embedding_size = 1024):
        """
        Keeps a reference to the provided RoBERTa model. 
        It then adds a linear layer that takes the distance between two 
        """
        super(WiC_Head, self).__init__()
        self.embedding_size = embedding_size
        self.embedder = roberta_based_model
        self.linear_diff = torch.nn.Linear(embedding_size, 250, bias = True)
        self.linear_seperator = torch.nn.Linear(250, 2, bias = True)
        self.loss = torch.nn.CrossEntropyLoss()
        self.activation = torch.nn.ReLU()
        self.softmax = torch.nn.Softmax()

    def forward(self, input_ids=None, attention_mask=None, labels=None,
                word1_locs = None, word2_locs = None):
        """
        Takes in the same argument as RoBERTa forward plus two tensors for the location of the 2 words to compare
        """
        if word1_locs is None or word2_locs is None:
          raise ValueError("The tensors (word1_locs, word1_locs) containing the location of the words to compare in the input vector must be provided.")
        elif input_ids is None:
          raise ValueError("The input_ids tensor must be provided.")
        elif word1_locs.shape[0] != input_ids.shape[0] or word2_locs.shape[0] != input_ids.shape[0]:
          raise ValueError("All provided vectors should have the same batch size.")
        batch_size = word1_locs.shape[0]
        embs, _ = self.embedder.roberta(input_ids=input_ids, attention_mask=attention_mask)
        word1s = torch.matmul(word1_locs, embs).view(batch_size, self.embedding_size)
        word2s = torch.matmul(word2_locs, embs).view(batch_size, self.embedding_size)
        diff = word1s - word2s
        layer1_results = self.activation(self.linear_diff(diff))
        logits = self.softmax(self.linear_seperator(layer1_results))
        outputs = logits
        # Losss
        if labels is not None:
            loss = self.loss(logits.view(-1, 2), labels.view(-1))
            outputs = (loss, logits)
        return outputs

## Get model

In [ ]:
class_model = WiC_Head(model, embedding_size = 768)

## The training loop

In [ ]:
from transformers import AdamW

In [ ]:
from transformers import AutoModelWithLMHead, AutoTokenizer

model = AutoModelWithLMHead.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")

inputs = tokenizer.encode("translate English to German: Hugging Face is a technology company based in New York and Paris", return_tensors="pt")
outputs = model.generate(inputs, max_length=40, num_beams=4, early_stopping=True)

print(outputs)

In [ ]:
tokenizer.decode(outputs)

In [ ]:
class_model.embedder.requires_grad_ = False

In [ ]:
print(class_model.embedder.requires_grad_)

In [ ]:

MIN_ACCURACY = 0.99 
REACHED_MIN_ACCURACY = False
best_weights = class_model.state_dict()
max_val_acc = (0, 0)
class_model.cuda()

param_optimizer = list(class_model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5)
fit_history = {"loss": [],  "accuracy": [], "val_loss": [], "val_accuracy": []}
epoch_number = 0
epoch_since_max = 0
continue_learning = True
while epoch_number < EPOCHS and continue_learning:
  epoch_number += 1
  print(f"Training epoch #{epoch_number}")
  tr_loss, tr_accuracy = 0, 0
  nb_tr_examples, nb_tr_steps = 0, 0
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0
  # Training
  
  class_model.train()
  # Freeze weights
  class_model.embedder.requires_grad_ = False
  #class_model.embedder.requires_grad_ = True
  for step, batch in enumerate(train_dataloader):
    batch = tuple(t.cuda() for t in batch)
    b_input_ids, b_token_ids, b_input_mask, b_labels, b_word1, b_word2, b_index = batch
    optimizer.zero_grad() 
    loss, logits = class_model(b_input_ids, attention_mask=b_input_mask, 
                               labels=b_labels, word1_locs = b_word1, word2_locs = b_word2) 
    loss.backward()

    optimizer.step()

    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.cpu().numpy()
    # Calculate the accuracy
    b_accuracy = flat_accuracy(logits, label_ids) # For RobertaForClassification
    # Append to fit history
    fit_history["loss"].append(loss.item()) 
    fit_history["accuracy"].append(b_accuracy) 
    # Update tracking variables
    tr_loss += loss.item()
    tr_accuracy += b_accuracy
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1
    if nb_tr_steps%10 == 0:
      print("\t\tTraining Batch {}: Loss: {}; Accuracy: {}".format(nb_tr_steps, loss.item(), b_accuracy))
  print("Training:\n\tLoss: {}; Accuracy: {}".format(tr_loss/nb_tr_steps, tr_accuracy/nb_tr_steps))
  # Validation
  class_model.eval()
  for batch in validation_dataloader:
    batch = tuple(t.cuda() for t in batch)
    b_input_ids, b_token_ids, b_input_mask, b_labels, b_word1, b_word2, b_index = batch
    with torch.no_grad():
      #loss, logits = class_model(b_input_ids, token_type_ids=b_token_ids, attention_mask=b_input_mask, labels=b_labels)
      loss, logits = class_model(b_input_ids, attention_mask=b_input_mask, 
                                 labels=b_labels, word1_locs = b_word1, word2_locs = b_word2)
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.cpu().numpy()
  
    b_accuracy = flat_accuracy(logits, label_ids)
    
    fit_history["val_loss"].append(loss.item()) 
    fit_history["val_accuracy"].append(b_accuracy) 
    # Update tracking variables
    eval_loss += loss.item()
    eval_accuracy += b_accuracy
    nb_eval_examples += b_input_ids.size(0)
    nb_eval_steps += 1
    if nb_eval_steps%10 == 0:
      print("\t\tValidation Batch {}: Loss: {}; Accuracy: {}".format(nb_eval_steps, loss.item(), b_accuracy))
  eval_acc = eval_accuracy/nb_eval_steps
  if eval_acc >= max_val_acc[0]:
    max_val_acc = (eval_acc, epoch_number)
    continue_learning = True
    epoch_since_max = 0 
    best_weights = copy.deepcopy(class_model.state_dict()) 
    if eval_acc >= MIN_ACCURACY:
      REACHED_MIN_ACCURACY = True
    if REACHED_MIN_ACCURACY:
      torch.save(best_weights, os.path.join(PATH,'WiCHead.pt'))
      continue_learning = False 
  else:
    epoch_since_max += 1
    if epoch_since_max > PATIENCE:
      continue_learning = False 
  print("Validation:\n\tLoss={}; Accuracy: {}".format(eval_loss/nb_eval_steps, eval_accuracy/nb_eval_steps))
print(f"Best accuracy ({max_val_acc[0]}) obtained at epoch #{max_val_acc[1]}.")

class_model.load_state_dict(best_weights)

In [ ]:
load_json_objects_from_file('../input/semeval/MCL-WiC/dev/multilingual/dev.ar-ar.data')

### Save fit history to json file

In [ ]:
with open(os.path.join(PATH, "fit_history.json"), 'w') as json_file:
  json.dump(fit_history, json_file)

### Save a model that was close to the baseline (Manual choice)

In [ ]:
# Save the best weights to file
# torch.save(best_weights, os.path.join(PATH,'WiCHead.pt'))

## Load an already trained WiC model from file

In [ ]:
# Load the model
#class_model.load_state_dict(torch.load(os.path.join(PATH,'WiCHead.pt')))
#class_model.cuda()

## Get the predictions on the validation set

In [ ]:
validation_predictions_correctness = {}
# Validation
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
# Put model in evaluation mode
class_model.eval()
# Evaluate data for one epoch
for batch in validation_dataloader:
  # Add batch to GPU
  batch = tuple(t.cuda() for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_token_ids, b_input_mask, b_labels, b_word1, b_word2, b_index = batch
  # Telling the model not to compute or store gradients, saving memory and speeding up validation
  with torch.no_grad():
    # Forward pass, calculate logit predictions
    #loss, logits = class_model(b_input_ids, token_type_ids=b_token_ids, attention_mask=b_input_mask, labels=b_labels)
    loss, logits = class_model(b_input_ids, attention_mask=b_input_mask, 
                                labels=b_labels, word1_locs = b_word1, word2_locs = b_word2)
  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.cpu().numpy()
  # Calculate the accuracy
  b_accuracy, b_pred_correctness = flat_accuracy(logits, label_ids, return_predict_correctness = True) # For RobertaForClassification
  indexes = b_index.detach().cpu().numpy() # Get the indexes
  # Add to predictions
  for index, pred in zip(indexes, b_pred_correctness):
    validation_predictions_correctness[index] = pred
  # Update tracking variables
  eval_loss += loss.item()
  eval_accuracy += b_accuracy
  nb_eval_examples += b_input_ids.size(0)
  nb_eval_steps += 1
  if nb_eval_steps%10 == 0:
    print("\t\tValidation Batch {}: Loss: {}; Accuracy: {}".format(nb_eval_steps, loss.item(), b_accuracy))
print("Validation:\n\tLoss={}; Accuracy: {}".format(eval_loss/nb_eval_steps, eval_accuracy/nb_eval_steps))
validation_predictions_correctness = collections.OrderedDict(sorted(validation_predictions_correctness.items()))
print(validation_predictions_correctness)

## Get the testing results
**Works for now. Would need to be modified to submit for competition reward.**

See https://competitions.codalab.org/competitions/20010#learn_the_details-evaluation

**Update:** The jsonl file from superglue is not in the same order as the text file taken from WiC. It doesn't match the idx parameter either. That is weird.

In [ ]:
test_predictions = {}
nb_test_examples, nb_test_steps = 0, 0
# Testing
# Put model in evaluation mode to evaluate loss on the validation set
class_model.eval()
# Evaluate data for one epoch
for batch in test_dataloader:
  # Add batch to GPU
  batch = tuple(t.cuda() for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_token_ids, b_input_mask, b_word1, b_word2, b_index = batch
  # Telling the model not to compute or store gradients, saving memory and speeding up validation
  with torch.no_grad():
    # Forward pass, calculate logit predictions
    logits = class_model(b_input_ids, attention_mask=b_input_mask, word1_locs = b_word1, word2_locs = b_word2)
  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  # Get the predictions
  b_preds = flat_predictions(logits)
  indexes = b_index.detach().cpu().numpy() # Get the indexes
  for index, pred in zip(indexes, b_preds):
    test_predictions[index] = pred
  # Update tracking variables
  nb_test_examples += b_input_ids.size(0)
  nb_test_steps += 1
  if nb_test_steps%10 == 0:
    print("\t\tTest Batch {}".format(nb_test_steps))
# Print final results
print("Testing Done!")
test_predictions = collections.OrderedDict(sorted(test_predictions.items()))
print(test_predictions)